In [1]:
import os
import sys

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import ml.generators.mip_generator as generator
import tensorflow as tf

/home/harold_triedman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [5]:
TrainGen = generator.MipGenerator
ValGen = generator.MipGenerator
train_gen = TrainGen(dims=(220, 220, 3),
                          batch_size=16,
                          augment_data=True,
                          extend_dims=False,
                          validation=False)
val_gen = ValGen(dims=(220, 220, 3),
                          batch_size=16,
                          augment_data=True,
                          extend_dims=False,
                          validation=True)

In [6]:
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import adam

In [7]:
def add_new_last_layer(base_model, nb_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(2048, activation='relu')(x)
    predictions = Dense(nb_classes, activation='sigmoid')(x)
    model = Model(input=base_model.input, output=predictions)
    return model

In [8]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(220, 220, 3))
for layer in base_model.layers:
    layer.trainable = False
finished_model = add_new_last_layer(base_model, 1)
finished_model.compile(optimizer=adam(lr=0.0001),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
finished_model.summary()

80142336/80134624 [==============================] - 14s 0us/step


ResourceExhaustedError: OOM when allocating tensor with shape[512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: block4_conv3/bias/Assign = Assign[T=DT_FLOAT, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block4_conv3/bias, block4_conv1/Const)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'block4_conv3/bias/Assign', defined at:
  File "/home/harold_triedman/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/harold_triedman/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/harold_triedman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/harold_triedman/anaconda3/lib/python3.6/site-packag

In [ ]:
mc_callback = ModelCheckpoint(filepath='tmp/vgg16_pretrained_weights.hdf5', verbose=1)
finished_model.fit_generator(
    generator=train_gen.generate(),
    steps_per_epoch=train_gen.get_steps_per_epoch(),
    validation_data=val_gen.generate(),
    validation_steps=val_gen.get_steps_per_epoch(),
    epochs=20,
    callbacks=[mc_callback],
    verbose=1,
    max_queue_size=1)